# NetNeighbors: Domain Discovery Using CommonCrawl Webgraph

**High-Performance JVM Backend**

Discover related domains using link topology analysis from the CommonCrawl web graph.

This notebook uses py4j to maintain a persistent JVM with the graph loaded in memory.
After initial load (~5 seconds), queries are **nearly instant**.

**Run the cells below in order to set up and use the discovery tool.**

In [1]:
# Step 1: Check RAM and setup working directory
import psutil
import os

ram_gb = psutil.virtual_memory().total / (1024**3)
print(f"Available RAM: {ram_gb:.1f} GB")

if ram_gb < 20:
    print("\n⚠️ WARNING: You need Colab Pro for this notebook!")
    print("   Required: 20GB+ RAM")
    print(f"   You have: {ram_gb:.1f} GB")
    print("\n   Please enable High-RAM runtime:")
    print("   Runtime → Change runtime type → Runtime shape: High-RAM")
    raise Exception("Insufficient RAM. Please upgrade runtime.")
else:
    print("✅ Sufficient RAM available\n")

# Determine NetNeighbors location and set as working directory
if os.path.exists("/content"):
    # Colab environment
    if not os.path.exists("/content/NetNeighbors"):
        print("Cloning NetNeighbors repository...")
        !git clone --depth 1 https://github.com/PeterCarragher/NetNeighbors.git /content/NetNeighbors > /dev/null 2>&1
        print("✅ Repository cloned")
    else:
        print("✅ NetNeighbors repository already exists")
    os.chdir("/content/NetNeighbors")
else:
    # Local environment
    if os.path.exists("src/DiscoveryTool.java"):
        print("✅ Already in NetNeighbors directory")
    elif os.path.exists("NetNeighbors/src/DiscoveryTool.java"):
        os.chdir("NetNeighbors")
        print("✅ Changed to NetNeighbors submodule")
    else:
        raise Exception("Cannot find NetNeighbors directory.")

print(f"Working directory: {os.getcwd()}")

Available RAM: 31.3 GB
✅ Sufficient RAM available

✅ Already in NetNeighbors directory
Working directory: /root/dev/NetNeighbors


### Step 2: Run Environment Setup

Installs Java 17, Maven, py4j, and builds the cc-webgraph tools.

In [2]:
!bash scripts/setup.sh

# Install py4j for JVM bridge
!pip install -q py4j
print("\n✅ py4j installed")

           NetNeighbors Environment Setup
Base directory: /root/dev
NetNeighbors: /root/dev/NetNeighbors
Mode: local

1. Setting up Java 17 and Maven...
   ✅ Java and Maven already installed
openjdk version "17.0.18" 2026-01-20

2. Skipping gcsfuse (local mode, not needed)

3. Installing Python dependencies...
   ✅ Python dependencies installed

4. Setting up cc-webgraph...
   ✅ cc-webgraph already built

5. Setting up NetNeighbors...
   ✅ DiscoveryTool already compiled

                    Setup Complete!

Next steps:
  1. Download webgraph data (use utils.download_webgraph)
  2. Run verify.sh to confirm installation

✅ py4j installed


### Step 3: Configure Storage and Download Webgraph

Downloads pre-built graph files from CommonCrawl (~23GB total).

In [3]:
from utils import setup_storage, download_webgraph

# Webgraph version - see https://commoncrawl.org/web-graphs for available versions
VERSION = "cc-main-2024-feb-apr-may"

# Enter GCS bucket name (or leave empty for local storage)
GCS_BUCKET = None #"commoncrawl_webgraph" # e.g., "my-webgraph-bucket"
LOCAL_PATH = "/mnt/d/dev/data/cc/"

if GCS_BUCKET:
  from google.colab import auth; auth.authenticate_user()
WEBGRAPH_DIR = setup_storage(bucket_name=GCS_BUCKET, webgraph_dir=LOCAL_PATH)

Using local storage: /mnt/d/dev/data/cc/


In [4]:
# Download webgraph files (skip if already downloaded)
download_webgraph(WEBGRAPH_DIR, VERSION)

Destination: /mnt/d/dev/data/cc/

Already exists: cc-main-2024-feb-apr-may-domain-vertices.txt.gz (889.2 MB)
Already exists: cc-main-2024-feb-apr-may-domain.graph (4298.4 MB)
Already exists: cc-main-2024-feb-apr-may-domain.properties (0.0 MB)
Already exists: cc-main-2024-feb-apr-may-domain-t.graph (4275.1 MB)
Already exists: cc-main-2024-feb-apr-may-domain-t.properties (0.0 MB)
Already exists: cc-main-2024-feb-apr-may-domain.stats (0.0 MB)

All graph files downloaded!

Building offset files (required for graph queries)...
Offsets already exist: cc-main-2024-feb-apr-may-domain.offsets
Offsets already exist: cc-main-2024-feb-apr-may-domain-t.offsets

Webgraph ready for use!


### Step 3b: Build Vertex Map

Builds the vertex label ↔ ID mapping required for graph exploration. This creates an ImmutableExternalPrefixMap (`.iepm`) that allows translating between domain names and their graph vertex IDs. **Skips automatically if already built.**

In [5]:
!bash scripts/build_vertex_map.sh {WEBGRAPH_DIR} {VERSION}

Building vertex map for cc-main-2024-feb-apr-may-domain...
  Graph directory: /mnt/d/dev/data/cc/
  cc-webgraph: /root/dev/cc-webgraph
  This may take several minutes...

Verifying webgraph files:
 .graph      : webgraph / BVGraph    (cc-main-2024-feb-apr-may-domain.graph)
 .properties : webgraph properties   (cc-main-2024-feb-apr-may-domain.properties)
Verifying webgraph files (transpose of the graph):
 .graph      : webgraph / BVGraph    (cc-main-2024-feb-apr-may-domain-t.graph)
 .properties : webgraph properties   (cc-main-2024-feb-apr-may-domain-t.properties)
The sort command supports parallel sort threads
+ /bin/java -Xmx25660m -cp /root/dev/cc-webgraph/src/script/webgraph_ranking/../../../target/cc-webgraph-0.1-SNAPSHOT-jar-with-dependencies.jar it.unimi.dsi.util.ImmutableExternalPrefixMap --block-size 4Ki cc-main-2024-feb-apr-may-domain.iepm

real	1m42.968s
user	1m49.002s
sys	0m24.659s
immutable external prefix map successfully built: cc-main-2024-feb-apr-may-domain.iepm

Prepar

### Step 4: Initialize Graph Bridge (JVM Backend)

This starts a persistent JVM and loads the graph into memory.
**Takes ~5 seconds**, but then all queries are nearly instant!

In [6]:
from graph_bridge import GraphBridge

# Initialize and load graph (this is the ~10 second step)
bridge = GraphBridge(WEBGRAPH_DIR, VERSION)
bridge.load_graph()

print("\n" + "="*60)
print("🚀 Graph loaded! Queries are now instant.")
print("="*60)

Starting JVM with cc-webgraph...
JAR: /root/dev/cc-webgraph/target/cc-webgraph-0.1-SNAPSHOT-jar-with-dependencies.jar
Loading graph: /mnt/d/dev/data/cc/cc-main-2024-feb-apr-may-domain
This takes ~5 seconds...
✅ Graph loaded!
Subsequent queries will be nearly instant!

🚀 Graph loaded! Queries are now instant.


### Step 5: Quick Test

Let's verify the bridge is working with a quick query.

In [7]:
import time

# Test domain lookup (should be instant)
test_domains = ["cnn.com", "bbc.com", "foxnews.com", "100percentfedup.com", "nonexistentdomain.tld", "4chan.org", "911truth.org"]

def reverse_domain(domain: str) -> str:
    return '.'.join(reversed(domain.split('.')))

reversed_domains = [reverse_domain(d) for d in test_domains]

start = time.time()
found, not_found = bridge.validate_seeds(reversed_domains)
elapsed = time.time() - start

print(f"Validated {len(test_domains)} domains in {elapsed*1000:.1f}ms")
print(f"Found: {found}")
if not_found:
    print(f"Not found: {not_found}")

Validated 7 domains in 18.6ms
Found: ['com.cnn', 'com.bbc', 'com.foxnews', 'com.100percentfedup', 'org.4chan', 'org.911truth']
Not found: ['tld.nonexistentdomain']


In [8]:
# Example: Direct API usage
import time

# Run discovery
threshold = 6
start = time.time()
results = bridge.discover_backlinks(found, min_connections=threshold)
# results = bridge.shared_predecessors(found)
elapsed = time.time() - start

print(f"\nDiscovery completed in {elapsed:.2f} seconds")
print(f"Found {len(results)} domains with >= {threshold} connections")
# print("\nTop 10 results:")
# for r in results[:10]:
#     print(f"  {r['domain']}: {r['connections']} connections ({r['percentage']}%)")
results

Processing 6 seed domains...
Processed 6/6 seeds...
Found 569,203 unique neighbor domains
Found 46 domains with >= 6 connections

Discovery completed in 0.93 seconds
Found 46 domains with >= 6 connections


[{'domain': 'com.50webs', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.aifsy', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.amgreatness', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.angelfire', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.bitchute', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.blogspot', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.dailycaller', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.ericpetersautos', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.fc2', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.forumotion', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.globalseoarticles', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.henrymakow', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.hubpages', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.kingranks', 'connections': 6, 'percentage': 100.0},
 {'domain': 'com.pirdu', 'conn

---

## Cleanup

When done, you can shutdown the JVM to free memory:

In [ ]:
# Uncomment to shutdown JVM
bridge.shutdown()
print("JVM shutdown complete")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39925)
Traceback (most recent call last):
  File "/home/peter/miniconda3/envs/net_neighbor/lib/python3.12/site-packages/py4j/java_gateway.py", line 982, in _get_connection
    connection = self.deque.pop()
                 ^^^^^^^^^^^^^^^^
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/peter/miniconda3/envs/net_neighbor/lib/python3.12/site-packages/py4j/java_gateway.py", line 1170, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


JVM shutdown complete
